# Debugging Jupyter Notebook for OCTolyzer's analysis of H-line/V-line single OCT B-scans

This notebook copies the step-by-step process of OCTolyzer's analysis pipeline for single H-line/V-line OCT B-scan analysis, and should provide the end-user with the means of debugging the pipeline by testing each step individually.

### Add OCTolyzer to system paths to permit access to analysis files

In [ ]:
import sys
from importlib import reload
sys.path.append(r'../')

### Import necessary packages and libraries

In [ ]:
import numpy as np
import os
import pandas as pd
import cv2
import scipy
import matplotlib.pyplot as plt
from octolyzer import utils, analyse, analyse_slo, main
from octolyzer.segment.octseg import choroidalyzer_inference, deepgpet_inference
from octolyzer.segment.sloseg import slo_inference, avo_inference, fov_inference
from octolyzer.measure.bscan.thickness_maps import grid, map as map_module
from octolyzer.measure.bscan import bscan_measurements, utils as chor_utils

### Define the file you want to use to debug, the `path` is the only variable to change

In [ ]:
KEY_LAYER_DICT = {"ILM": "Inner Limiting Membrane",
                  "RNFL": "Retinal Nerve Fiber Layer",
                  "GCL": "Ganglion Cell Layer",
                  "IPL": "Inner Plexiform Layer",
                  "INL": "Inner Nuclear Layer",
                  "OPL": "Outer Plexiform Layer",
                  "ELM": "External Limiting Membrane", # Outer nuclear layer
                  "PR1": "Photoreceptor Layer 1",
                  "PR2": "Photoreceptor Layer 2",
                  "RPE": "Retinal Pigment Epithelium",
                  "BM": "Bruch's Membrane Complex", 
                  "CHORupper": "Bruch's Membrane - Choroid boundary",
                  "CHORlower":"Choroid - Sclera boundary"}

In [ ]:
path = r"../demo/input/Ppole_1.vol"

In [ ]:
fname = os.path.split(path)[1].split(".")[0]

save_path = f"check_ppole/{fname}"
segmentation_directory = f"check_ppole/oct_segmentations"

if not os.path.exists(os.path.split(save_path)[0]):
    os.mkdir(os.path.split(save_path)[0])

if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(segmentation_directory):
    os.mkdir(segmentation_directory)

## Set the relevant configuration parameters in `param_dict` to whatever is relevant to your analysis

In [ ]:
param_dict = {
    "save_individual_segmentations": 1,
    "save_individual_images": 1,
    "preprocess_bscans": 1,
    "analyse_choroid": 1,
    "analyse_slo": 1,
    "custom_maps": ['ILM_OPL'], # this cannot be "0" like it is in config.txt - it is an empty list
    "analyse_all_maps": 1,
    "analyse_square_grid": 1,
    "choroid_measure_type": "perpendicular",
    "linescan_roi_distance": 3000
}

# flags for choroid analysis, preprocessing bscans
preprocess_data = param_dict["preprocess_bscans"]

# For saving out representative Bscan/SLO/segmentation masks
save_ind_segmentations = param_dict["save_individual_segmentations"]
save_ind_images = param_dict["save_individual_images"]

# Custom retinal thickness maps
custom_maps = param_dict["custom_maps"]
all_maps = param_dict["analyse_all_maps"]

# analysing choroid?
analyse_choroid = param_dict['analyse_choroid']

# square grid for Ppole
sq_grid_flag = param_dict['analyse_square_grid']

# analysing SLO?
analyse_slo_flag = param_dict['analyse_slo']

# User-specified measure type for choroid
chor_measure_type = param_dict['choroid_measure_type']

# User-specified ROI distance either side of fovea
macula_rum = param_dict['linescan_roi_distance']

In [ ]:
# By default we save individual results and collate segmentations
collate_segmentations = 1
have_slo = 1 # Assume we have the SLO as these are .vol files

# Default bscan/slo measurement parameters
N_measures = "all" # Measuring all thicknesses across ROI to average over
N_avgs = 0 # Robust thickness estimation, only relevant when N_measures is an integer
chor_linescan_measure_type = chor_measure_type # Measuring type for choroidal metrics across OCT Linescans
chor_ppole_measure_type = chor_measure_type # Measuring type for choroidal metrics across OCT Volumes
ret_measure_type = 'vertical' # Measuring retina column-wise (via A-scans) according to most devices/literature

In [ ]:
# Default parameters for thickness maps: ETDRS grid and optional square grid
etdrs_kwds = {"etdrs_microns":[1000,3000,6000]}
square_kwds = {"N_grid":8, "grid_size":7000}
map_flags = [1, sq_grid_flag]
map_kwds = [etdrs_kwds, square_kwds]

### Load in data from `.vol` file

In [ ]:
verbose=1
oct_output = []
logging_list=[]
output = utils.load_volfile(path, preprocess=preprocess_data*analyse_choroid, verbose=verbose,
                            custom_maps=custom_maps, logging=logging_list)
bscan_data, metadata, slo_output, layer_pairwise, logging_list = output
(slo, slo_acq_fixed, slo_acq, (slo_pad_x, slo_pad_y)) = slo_output
slo_pad_xy = np.array([slo_pad_x[0], slo_pad_y[0]])
N_scans, M, N = bscan_data.shape
slo_N = slo.shape[0]
oct_output.append(bscan_data)

# Pixel spacing, SLO pixel scaling is assumed as isotropic
scaleX, scaleY, scaleZ = metadata["bscan_scale_x"],metadata["bscan_scale_y"],metadata["bscan_scale_z"]
bscan_scale = (scaleX, scaleY)
bscan_ROI = metadata["bscan_ROI_mm"]
slo_scale = metadata["slo_scale_xy"]

# Analyse the SLO image
scan_type = metadata["bscan_type"]
scan_location = metadata["location"]
if scan_location == "peripapillary":
    slo_location = "Optic disc"
else:
    slo_location = "Macula"
eye = metadata["eye"]

output[1]

In [ ]:
# Detect retinal layer keys
pairwise_keys = list(layer_pairwise.keys())
layer_keys = list(set(pd.DataFrame(pairwise_keys).reset_index(drop=True)[0].str.split("_", expand=True).values.flatten()))

### Load in SLO models and analyse SLO (assuming the SLO analysis is *not* where the error is coming from)

In [ ]:
slo_metrics = False
segmentation_dict = {}
slo_model = slo_inference.SLOSegmenter()
fov_model = fov_inference.FOVSegmenter()
avo_model = avo_inference.AVOSegmenter()
slo_analysis_output = analyse_slo.analyse(255*slo, save_path,
                                        slo_scale, slo_location, eye,
                                        slo_model, avo_model, fov_model,
                                        save_images=save_ind_segmentations, 
                                        compute_metrics=slo_metrics, verbose=verbose, 
                                        collate_segmentations=True, segmentation_dict=segmentation_dict)
slo_meta_df, slo_measure_dfs, _, slo_segmentations, slo_logging_list = slo_analysis_output
slo_missing_fovea = slo_meta_df.slo_missing_fovea.values[0].astype(bool)
logging_list.extend(slo_logging_list)
slo_avimout = slo_segmentations[-1]

### Load in choroid segmentation models and segment (if `analyse_choroid` is 1)

In [ ]:
# Alert to user we are analysing OCT from here on
msg = f"\n\nANALYSING OCT of {fname}.\n"
logging_list.append(msg)
if verbose:
    print(msg)

choroidalyzer = choroidalyzer_inference.Choroidalyzer()
deepgpet = deepgpet_inference.DeepGPET()

if analyse_choroid:
    msg = "Segmenting choroid and fovea..."
else:
    msg = "Detecting fovea for grid/ROI alignment (through use of Choroidalyzer)..."
logging_list.append(msg)
if verbose:
    print(msg)
    
# If macula-centred, use Choroidalyzer. If optic disc-centred, use deepGPET  
if scan_location == "macular":
    if N_scans == 1 or choroidalyzer.device == 'cpu':
        rvfmasks, foveas, fov_scores = choroidalyzer.predict_list(bscan_data, soft_pred=True)
    else:
        rvfmasks, foveas, fov_scores = choroidalyzer.predict_batch(bscan_data, soft_pred=True)
elif scan_location == "peripapillary":
    rvfmasks = deepgpet.predict_list(bscan_data, soft_pred=True)

# Resolve fovea detection. If at origin then threshold too high, apply filter function and warn user.
if scan_location != "peripapillary":
    # Method 1: default to middle of stack, unreliable due to poor acquisition but mostly correct
    # fovea_slice_num = N_scans//2 
    
    # Method 2: detect fovea based on the highest score from Choroidalyzer, unreliable due to poor segmentation but mostly correct.
    if scan_type == 'Ppole':
        fovea_slice_num = int(fov_scores.argmax(axis=0)[0])
    else:
        fovea_slice_num = N_scans//2 
    
    # Extract fovea from list using fovea_slice_num
    fovea = foveas[fovea_slice_num]

### Organise segmentations for map generation

In [ ]:
msg = f"""\nGenerating thickness and volume maps following ETDRS (0.5mm,1.5mm,3mm radial concentric grids).
All retinal measurements are made vertically, i.e. with respect to the image axis (vertical).
All choroidal measurements are made {chor_measure_type}.
NOTE: Subregion volumes will not be computed for CVI map."""
logging_list.append(msg)
if verbose:
    print(msg)

# Extract parameters for generating maps, rmove any vessel pixels outside choroid region for vmasks
if analyse_choroid:
    
    # Error handling for unexpected issues in volume stack when post-processing choroid segmentations
    rmasks = []
    rtraces = []
    vmasks = []
    for i, rvf_i in enumerate(rvfmasks):
        try:
            trace = utils.get_trace(rvf_i[0], 0.5, align=False)
            rtraces.append(trace)
            rmasks.append(utils.rebuild_mask(trace, img_shape=(M,N)))
        except:
            rtraces.append((-1*np.ones((N,2)), -1*np.ones((N,2))))
            rmasks.append(np.zeros((M, N)))
    rmasks = np.array(rmasks)
    vmasks = np.array([rmask*rvf_i[1] for (rmask, rvf_i) in zip(rmasks, rvfmasks)])

# By default setup default choroid and retinal maps.
if analyse_choroid:
    ppole_keys = ["choroid", "choroid_vessel", 'ILM_BM']
    ppole_units = ['[um]', '[um2]', '[um]']
    ppole_segs = [rmasks, rmasks, layer_pairwise['ILM_BM']]
else:
    ppole_keys = ['ILM_BM']
    ppole_units = ['[um]']
    ppole_segs = [layer_pairwise['ILM_BM']]

# If retina fully segmentd, then we can also extract other custom_maps.
if len(layer_pairwise) > 1:
    if all_maps:
        for key_pair in pairwise_keys:
            if key_pair not in custom_maps:
                ppole_keys.append(key_pair)
                ppole_units.append('[um]')
                ppole_segs.append(layer_pairwise[key_pair])
    if len(custom_maps) > 0:
        for key_pair in custom_maps:
            if key_pair not in ppole_keys:
                ppole_keys.append(key_pair)
                ppole_units.append('[um]')
                ppole_segs.append(layer_pairwise[key_pair])


### Rename layer and maps and initialise measurement dictionaries 

In [ ]:
# Rename summary layers
keys_to_names = ['ILM_BM', 'ILM_ELM', 'ELM_BM']
names_to_keys = ['retina', 'inner_retina', 'outer_retina']
ppole_keys = np.array(ppole_keys).astype('<U14')
for k2n, n2k in zip(keys_to_names, names_to_keys):
    ppole_keys[ppole_keys==k2n] = n2k
ppole_keys = list(ppole_keys)

# Initialise dictionaries to store maps and feature measurements from volume scans
grid_type = ["etdrs", "square"]
map_dict = {}
measure_dict = {}
volmeasure_dict = {}
if collate_segmentations:
    ctmap_args = {}
    ctmap_args['core'] = [slo, fname, segmentation_directory]
for (m_flag, m_type) in zip(map_flags, grid_type):
    if m_flag:
        measure_dict[m_type] = {}
        volmeasure_dict[m_type] = {}

# save out thickness maps and visualisations in single folder to clean up directory
map_save_path = os.path.join(save_path,'thickness_maps')
if not os.path.exists(map_save_path):
    os.mkdir(map_save_path)

### Compute maps and measure ETDRS/Square etc.

In [ ]:
# Loop over segmented layers and generate user-specified maps
for key, seg in zip(ppole_keys, ppole_segs):

    # Log to user and take special care for choroid_vessel map
    msg = f"    {key} thickness map"
    ves_chorsegs = None
    measure_type = "vertical"
    if "choroid" in key:
        measure_type = ret_measure_type
        if key == "choroid_vessel":
            ves_chorsegs = vmasks
            measure_type = chor_ppole_measure_type
            msg = f"    choroid vessel and vascular index maps"                    

    # Compute map
    logging_list.append(msg)
    if verbose:
        print(msg)
    map_output = map_module.construct_map(slo, 
                                          slo_acq,
                                          slo_pad_xy,
                                          seg,
                                          fovea, 
                                          fovea_slice_num, 
                                          bscan_scale, 
                                          scaleZ,
                                          slo_N=slo_N, 
                                          oct_N=N,
                                          log_list=[],
                                          ves_chorsegs=ves_chorsegs,
                                          measure_type=measure_type)

    # Measure grids on the maps and save out in dedicated folder 
    for i,(m_flag, m_kwd) in enumerate(zip(map_flags, map_kwds)):

        # If flagged to measure ETDRS/Posterior pole grid then allow grid measurement
        m_type = grid_type[i]
        if m_flag:

            # For 'choroid_vessel', first measure CVI map with floats as CVI in [0,1]
            if key == "choroid_vessel":
                slo_output, macular_map, (angle, fovea_at_slo, acq_centre), cvi_map, map_messages = map_output
                logging_list.extend(map_messages)
                cvi_key = "choroid_CVI"
                ppole_units.append('')
                ppole_keys.append(cvi_key)
                fname_key = fname+f"_{cvi_key}_{m_type}_map"

                # CVI-specific grid measurement
                dtype = np.float64
                grid_measure_output = grid.measure_grid(cvi_map, 
                                                        fovea_at_slo, 
                                                        scaleX, 
                                                        eye, 
                                                        rotate=angle, 
                                                        measure_type=m_type, 
                                                        grid_kwds=m_kwd,
                                                        interp=True, 
                                                        plot=save_ind_segmentations, 
                                                        slo=slo_output, 
                                                        dtype=dtype,
                                                        fname=fname_key, 
                                                        save_path=map_save_path)
                grid_output, gridvol_output, grid_messages = grid_measure_output

                # Append results to dictionaries
                logging_list.extend(grid_messages)                                
                measure_dict[m_type][cvi_key] = grid_output
                volmeasure_dict[m_type][cvi_key] = gridvol_output
                map_dict[cvi_key] = pd.DataFrame(cvi_map)

                # Necessary for visualisation
                if m_type=='etdrs' and collate_segmentations:
                    ctmap_args[cvi_key] = [cvi_map, 
                                           fovea_at_slo, 
                                           scaleX, 
                                           eye, 
                                           angle, 
                                           dtype,
                                           grid_output, 
                                           gridvol_output]
            
            else:

                # Standard output from constructing macular map when key != 'choroid_vessel'
                slo_output, macular_map, (angle, fovea_at_slo, acq_centre), map_messages = map_output
                logging_list.extend(map_messages)

            # Measure grid for all other metrics and layers other than CVI
            dtype = np.uint64
            unit = 'thickness' if m_type != 'choroid_vessel' else 'area'
            grid_measure_output = grid.measure_grid(macular_map, 
                                                    fovea_at_slo, 
                                                    scaleX, 
                                                    eye, 
                                                    rotate=angle, 
                                                    measure_type=m_type, 
                                                    grid_kwds=m_kwd,
                                                    interp=True, 
                                                    plot=save_ind_segmentations, 
                                                    slo=slo_output, 
                                                    dtype=dtype,
                                                    fname=fname+f"_{key}_{m_type}_{unit}_map", 
                                                    save_path=map_save_path)
            grid_output, gridvol_output, grid_messages = grid_measure_output

            # Append results to dictionaries
            logging_list.extend(grid_messages)                                                                 
            measure_dict[m_type][key] = grid_output
            volmeasure_dict[m_type][key] = gridvol_output

        # Append results to dictionaries
        map_dict[key] = pd.DataFrame(macular_map)
        if m_type=='etdrs' and key in ['retina','choroid'] and collate_segmentations:
            ctmap_args[key] = [macular_map, 
                               fovea_at_slo, 
                               scaleX, 
                               eye, 
                               angle, 
                               dtype,
                               grid_output, 
                               gridvol_output]

# Log to user that maps are being saved out
msg = f'Saving out key macular maps.'
logging_list.append(msg)
if verbose:
    print(msg)

# Plot core maps (retina, choroid, CVI) into single figure and save out
if collate_segmentations and map_flags[0]==1:
    fig = grid.plot_multiple_grids(ctmap_args)
    fig.savefig(os.path.join(save_path, fname+'.png'), bbox_inches="tight", transparent=False)
plt.close()

# Save out macular maps as .npy files 
for key, macular_map in map_dict.items():
    unit = ''
    if key != 'choroid_CVI':
        unit = 'thickness' if m_type != 'choroid_vessel' else 'area'
    np.save(os.path.join(map_save_path, f"{fname}_{key}_{unit}_map.npy"), macular_map)

# Add choroid Ppole traces to retinal segmentations
if analyse_choroid:
    layer_pairwise["CHORupper_CHORlower"] = rtraces
    layer_keys.append("CHORupper")
    layer_keys.append("CHORlower")

### Save out B-scans with segmentations overlaid

In [ ]:
# Save out volumetric OCT B-scan segmentations
if save_ind_segmentations:

    msg = f'Saving out key visualisations of segmentations overlaid onto posterior pole B-scans.'
    logging_list.append(msg)
    if verbose:
        print(msg)

    # Save out fovea-centred B-scan segmentation visualisation
    if analyse_choroid:
        fovea_vmask = vmasks[fovea_slice_num]
        fovea_vcmap = np.concatenate([fovea_vmask[...,np.newaxis]] 
                + 2*[np.zeros_like(fovea_vmask)[...,np.newaxis]] 
                + [fovea_vmask[...,np.newaxis] > 0.01], axis=-1)
    else:
        vmasks = None# Plot segmentations over fovea-centred B-scan

    layer_keys_copied = layer_keys.copy()
    fig, (ax0,ax) = plt.subplots(1,2,figsize=(12,6))
    ax0.imshow(bscan_data[fovea_slice_num], cmap='gray')
    ax.imshow(bscan_data[fovea_slice_num], cmap="gray")
    for key, tr in layer_pairwise.items():
        for (k, t) in zip(key.split("_"), tr[fovea_slice_num]):
            if k in layer_keys_copied:
                ax.plot(t[:,0],t[:,1], label='_ignore', zorder=2)
                layer_keys_copied.remove(k)
    ax.scatter(fovea[0], fovea[1], s=200, marker="X", edgecolor=(0,0,0), 
                color="r", linewidth=1, zorder=3, label='Detected fovea position')
    if analyse_choroid:
        ax.imshow(fovea_vcmap, alpha=0.5, zorder=2)
    ax.axis([0, N-1, M-1, 0])
    ax.legend(fontsize=16)
    ax.set_axis_off()
    ax0.set_axis_off()
    fig.tight_layout(pad = 0)
    fig.savefig(os.path.join(save_path, f"{fname}_fovea_octseg.png"), bbox_inches="tight")
    plt.close()
    
    # Stitch all B-scans to create "contact sheet" for checking
    # Organise stacking of B-scans into rows & columns
    if N_scans in [61,31,45,7]:
        if N_scans == 61:
            reshape_idx = (10,6)
        elif N_scans == 31:
            reshape_idx = (5,6)
        elif N_scans == 45:
            reshape_idx = (11,4)
        elif N_scans == 7:
            reshape_idx = (2,3)
        utils.plot_composite_bscans(bscan_data, 
                                    vmasks, 
                                    fovea_slice_num, 
                                    layer_pairwise, 
                                    reshape_idx, 
                                    analyse_choroid, 
                                    fname, 
                                    save_path)
    else:
        msg = f'Volume scan with {N_scans} B-scans cannot currently be reshaped into single composite image.\nThis is likely because the development team has not had access to this kind of volume scans before. Please raise an issue on the GitHub repository.'
        logging_list.append(msg)
        if verbose:
            print(msg)

In [ ]:
# Ppole measurement metadata
metadata["bscan_fovea_x"] = fovea[0]
metadata["bscan_fovea_y"] = fovea[1]
metadata["slo_fovea_x"] = fovea_at_slo[0]
metadata["slo_fovea_y"] = fovea_at_slo[1]
metadata["slo_missing_fovea"] = slo_missing_fovea
metadata["acquisition_angle_degrees"] = angle
metadata["choroid_measure_type"] = chor_measure_type

# Add metric units to the end of metadata
metadata["thickness_units"] = "microns"
metadata["choroid_vascular_index_units"] = 'dimensionless'
metadata["choroid_vessel_density_units"] = "micron2"
metadata["area_units"] = "mm2"
metadata["volume_units"] = "mm3"

In [ ]:
# If saving out bscan and slo image. If ppole, only saying out bscan at fovea
# This is automatically done for AV-line scans.
if save_ind_images:
    if have_slo:
        cv2.imwrite(os.path.join(save_path,f"{fname}_slo.png"), 
                    (255*slo).astype(np.uint8))
    if scan_location != 'peripapillary':
        cv2.imwrite(os.path.join(save_path,f"{fname}_slo_acquisition_lines.png"), 
                    (255*slo_acq).astype(np.uint8))
        cv2.imwrite(os.path.join(save_path,f"{fname}_bscan_fovea.png"), 
                (255*bscan_data[fovea_slice_num]).astype(np.uint8))
    else:
        cv2.imwrite(os.path.join(save_path,f"{fname}_bscan.png"), 
                (255*bscan_data[0]).astype(np.uint8))

### Organise segmentations to be saved out

In [ ]:
# Layer keys, ordered anaomtically
ordered_keys = np.array(list(KEY_LAYER_DICT))
key_df = pd.DataFrame({"key":layer_keys,"layer":[KEY_LAYER_DICT[key] for key in layer_keys],
                    "layer_number":[np.where(key == ordered_keys)[0][0] for key in layer_keys]})
key_df = key_df.sort_values("layer_number")
del key_df["layer_number"]

In [ ]:
# Organise layer segmentations to be saved out - overcomplicated as I am working
# with pairwise segmentation traces, not individual ones. 
seg_df = {}
layer_keys_copied = layer_keys.copy()
for key, trace_xy_all in layer_pairwise.items():
    for k_idx, k in enumerate(key.split("_")):
        if k in layer_keys_copied:
            all_ytr = {}
            for s_idx, trace in enumerate(trace_xy_all):
                t = trace[k_idx]
                (xtr, ytr) = t[:,0], t[:,1]
                try:
                    xst, xen = xtr[[0,-1]]
                    ytr_pad = np.pad(ytr, ((max(xst-1,0), N-xen)), mode="constant")
                    all_ytr[s_idx] = {i:ytr_pad[i] for i in range(N)}
                except Exception as e:
                    message = f"\nAn exception of type {type(e).__name__} occurred. Error description:\n{e.args[0]}"
                    user_fail = f"Failed to store segmentations for B-scan {s_idx+1}/{N_scans} for layer {k}. Saving as NAs"
                    log_save = [message, user_fail]
                    logging_list.extend(log_save)
                    if verbose:
                        print(message)
                        print(user_fail)
                    all_ytr[s_idx] = {i:np.nan for i in range(N)}

            layer_keys_copied.remove(k)
            df = utils.nested_dict_to_df(all_ytr).reset_index()
            df = df.rename({"index":"scan_number"}, inplace=False, axis=1)
            seg_df[k] = df

### Organise measurements into DataFrames

In [ ]:
# Organise feature measurements for Ppole volume scans
ppole_key_unit_df = pd.DataFrame({'map_name':ppole_keys, 'units':ppole_units}).drop_duplicates()
ppole_vol_unit_df = ppole_key_unit_df.copy()
ppole_vol_unit_df['units'] = '[mm3]'

# Extract only retinal layers
retina_layers = np.array(list(KEY_LAYER_DICT.keys())[:-2])
pairwise_keys = [f"{k1}_{k2}" for (k1,k2) in zip(retina_layers[:-1], retina_layers[1:])]
all_maps = ppole_key_unit_df.map_name.values

# Order rows of results dataframes anatomically
if analyse_choroid:
    choroid_maps = ['choroid', 'choroid_CVI', 'choroid_vessel']
else:
    choroid_maps = []
retina_sum_maps = []
retina_custom_maps = []
retina_layer_maps = []
for map_name in all_maps:
    if 'retina' in map_name:
        retina_sum_maps.append(map_name)
    elif 'choroid' not in map_name:
        if map_name in pairwise_keys:
            retina_layer_maps.append(map_name)
        else:
            retina_custom_maps.append(map_name)
ordered_maps = retina_sum_maps+retina_layer_maps+retina_custom_maps+choroid_maps

# Collect grid thickness/volume measurements in DataFrames
measure_dfs = []
measure_grids = []
volmeasure_dfs = []
for grid_type in ["etdrs", "square"]:
    if grid_type in measure_dict.keys():
        measure_grids.append(grid_type)

        # Unpack dict of dicts
        df = measure_dict[grid_type]
        df = utils.nested_dict_to_df(df).reset_index()
        df = df.rename({"index":"map_name"}, inplace=False, axis=1)
        df = df.merge(ppole_key_unit_df, on='map_name', how='inner')
        # add unit column and shift
        cols = list(df.columns)
        cols.insert(1, cols.pop(cols.index('units')))
        df = df.loc[:, cols]
        # Order rows anatomically
        df['map_name'] = pd.CategoricalIndex(df['map_name'], ordered=True, categories=ordered_maps)
        df = df.sort_values('map_name').reset_index(drop=True)
        measure_dfs.append(df.drop_duplicates())

        # Same for volume dataframes
        voldf = volmeasure_dict[grid_type]
        voldf = utils.nested_dict_to_df(voldf).reset_index()
        voldf = voldf.rename({"index":"map_name"}, inplace=False, axis=1)
        voldf = voldf.merge(ppole_vol_unit_df, on='map_name', how='inner')
        cols = list(voldf.columns)
        cols.insert(1, cols.pop(cols.index('units')))
        voldf = voldf.loc[:, cols]
        voldf['map_name'] = pd.CategoricalIndex(voldf['map_name'], ordered=True, categories=ordered_maps)
        voldf = voldf.sort_values('map_name').reset_index(drop=True)
        volmeasure_dfs.append(voldf.drop_duplicates())

### Save out results

In [ ]:
# Save out core results in an .xlsx file
meta_df = pd.DataFrame(metadata, index=[0])
with pd.ExcelWriter(os.path.join(save_path, f'{fname}_output.xlsx')) as writer:
    
    # Write metadata
    meta_df.to_excel(writer, sheet_name='metadata', index=False)

    # Save out metadata key and descriptions
    metakeydf = utils.metakey_df
    metakeydf = metakeydf[metakeydf.column.isin(list(meta_df.columns))]
    metakeydf.to_excel(writer, sheet_name='metadata_keys', index=False)

    # Write OCT results, either map measurements (for PPole only) or H-line/V-line/Radial measurements
    if scan_type == "Ppole":
        for measure_df, volmeasure_df, grid_type in zip(measure_dfs, volmeasure_dfs, measure_grids):
            measure_df.to_excel(writer, sheet_name=f'{grid_type}_measurements', index=False)
            volmeasure_df.to_excel(writer, sheet_name=f'{grid_type}_volume_measurements', index=False)    
    elif scan_type != "AV-line":
        for measure_df in measure_dfs:
            measure_df.to_excel(writer, sheet_name="oct_measurements", index=False)

    # Write SLO measurements
    if slo_analysis_output is not None and analyse_slo:
        for df in slo_measure_dfs:
            if len(df) > 0:
                z = df.zone.iloc[0]
                df.to_excel(writer, sheet_name=f'slo_measurements_{z}', index=False)

    # Write out segmentations
    for key,df in seg_df.items():
        if key != key.lower():
            name = f"segmentations_{key}"
        else:
            name = f"maps_{key}"
        df.to_excel(writer, sheet_name=name, index=False)

    # write out layer keys
    key_df.to_excel(writer, sheet_name="layer_keys", index=False)

msg = f"\nSaved out metadata, measurements and segmentations."
logging_list.append(msg)
if verbose:
    print(msg)

# Organise outputs from analysis script
oct_measures = measure_dfs.copy()
oct_segmentations_maps = [seg_df]
if scan_location != 'peripapillary':
    oct_segmentations_maps.append(vmasks)
if scan_type == "Ppole":
    oct_segmentations_maps.append(map_dict)
    for df in volmeasure_dfs:
        oct_measures.append(df)
    if sq_grid_flag:
        oct_measures = [oct_measures[0],oct_measures[2],oct_measures[1],oct_measures[3]]
oct_analysis_output = [meta_df, slo, bscan_data] + [oct_measures] + oct_segmentations_maps + [logging_list]

# final log
msg = f"\nCompleted analysis of {fname}.\n"
logging_list.append(msg)
if verbose:
    print(msg)

# Save out log
with open(os.path.join(save_path, f"{fname}_log.txt"), "w") as f:
    for line in logging_list:
        f.write(line+"\n")

# Run OCTolyzer's pipeline from scratch on this file to check whether it works properly before/after debugging. 

In [ ]:
analyse.analyse(path, save_path="check_ppole", param_dict=param_dict)